In [ ]:
"Multi face mask detection

In [1]:
# import the necessary packages
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import imutils
import numpy as np
import argparse
import os
from utils import *

In [2]:
# Load the model

json_file = open('model_3class.json', 'r')
loaded_model_json = json_file.read()
json_file.close()# load json and create model

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model_3class.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
cfg_path = 'weights/yolov3-face.cfg'
weights_path = 'weights/yolov3-wider_16000.weights'

[INFO] loading face detector model...


In [5]:
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [8]:
#Load the image you want to test
image = cv2.imread('inputs/images/pic2.jpg')
#image = cv2.imread('inputs/images/img5.jpeg')
orig = image.copy()

    
# Create a 4D blob from a frame.
blob = cv2.dnn.blobFromImage(image, 1 / 255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False)

# Sets the input to the network
net.setInput(blob)

# Runs the forward pass to get output of the output layers
outs = net.forward(get_outputs_names(net))

# Remove the bounding boxes with low confidence
faces = post_process(image, outs, CONF_THRESHOLD, NMS_THRESHOLD)
print('[i] ==> # detected faces: {}'.format(len(faces)))
print('#' * 60)

# initialize the set of information we'll displaying on the frame
info = [
    ('number of faces detected', '{}'.format(len(faces)))
]

for face in faces:
    left, top, width, height = face[0], face[1], face[2], face[3]
    startX, startY, endX, endY = left, top, left+width, top+height
    face = image[startY:endY, startX:endX]
    #face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, (224, 224))
    face = img_to_array(face)
    face = preprocess_input(face)
    face = np.expand_dims(face, axis=0)

    # pass the face through the model to determine if the face
    # has a mask or not
    (notCorrect,mask, withoutMask ) = model.predict(face)[0]

    # determine the class label and color we'll use to draw
    # the bounding box and text
    if (mask < 0.90 and withoutMask < 0.90) or notCorrect > max(mask, withoutMask):
            label, color = "Not correct", (0, 255, 255)
    elif mask > max (withoutMask, notCorrect):
        label, color = "Mask", (0, 255,0)
    elif withoutMask > max (mask, notCorrect):
        label, color = "No Mask", (0, 0, 255)
    # label = "Mask" if mask > max (withoutMask, notCorrect) else "No Mask"
    # color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

    # include the probability in the label
    label = "{}: {:.2f}%".format(label, max(notCorrect, mask, withoutMask) * 100)

    # display the label and bounding box rectangle on the output
    # frame
    cv2.putText(image, label, (startX, startY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

for (i, (txt, val)) in enumerate(info):
    text = '{}: {}'.format(txt, val)
    cv2.putText(image, text, (10, (i * 20) + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_RED, 2)

#save the output image
cv2.imwrite('outputs/output_multi.jpg', image)

cv2.imshow('', image)
cv2.waitKey(0)

cv2.destroyAllWindows()


[i] ==> # detected faces: 4
############################################################


In [ ]:
"""" Application to a video """"

In [6]:
size = 4
vs = cv2.VideoCapture('inputs/video/vid.mp4') 

#out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))


# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
    ret, frame = vs.read()

    #frame = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))
    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(get_outputs_names(net))

    # Remove the bounding boxes with low confidence
    faces = post_process(frame, outs, CONF_THRESHOLD, NMS_THRESHOLD)
    print('[i] ==> # detected faces: {}'.format(len(faces)))
    print('#' * 60)

    # initialize the set of information we'll displaying on the frame
    info = [
        ('number of faces detected', '{}'.format(len(faces)))
    ]

    for face in faces:
        left, top, width, height = face[0], face[1], face[2], face[3]
        startX, startY, endX, endY = left, top, left+width, top+height
        #(startX, startY) = (max(0, startX), max(0, startY))
        #(endX, endY)  = (min(w - 1, endX), min(h -1, endY))
        face = frame[startY:endY, startX:endX]
        #face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        #face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # pass the face through the model to determine if the face
        # has a mask or not
        (notCorrect, mask, withoutMask ) = model.predict(face)[0]

        # determine the class label and color we'll use to draw
        # the bounding box and text
        if (mask < 0.70 and withoutMask < 0.70) or notCorrect > max(mask, withoutMask):
                label, color = "Not correct", (0, 255, 255)
        elif mask > max (withoutMask, notCorrect):
            label, color = "Mask", (0, 255,0)
        elif withoutMask > max (mask, notCorrect):
            label, color = "No Mask", (0, 0, 255)
        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    for (i, (txt, val)) in enumerate(info):
        text = '{}: {}'.format(txt, val)
        cv2.putText(frame, text, (10, (i * 20) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_RED, 2)
    
    key = cv2.waitKey(10)
    if cv2.waitKey(1) & key==27:
        break
    # show the output frame
    cv2.namedWindow("LIVE", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("LIVE",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
    cv2.imshow("LIVE", frame)
    key = cv2.waitKey(10)
    
    # create an output video by appending each frame to it
    #out.write(cv2.resize(frame, (1040,880) ))
    
# Stop video
vs.release()

# Close all started windows
cv2.destroyAllWindows()


[i] ==> # detected faces: 3
############################################################
[i] ==> # detected faces: 4
############################################################
[i] ==> # detected faces: 4
############################################################
[i] ==> # detected faces: 4
############################################################
[i] ==> # detected faces: 4
############################################################


In [ ]:
""""" Application to your webcam """""

In [7]:
cap = cv2.VideoCapture(0) 

while True:

    ret, frame = cap.read()

    if not ret:
        cv2.waitKey(1000)
        break

    frame = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))
    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(get_outputs_names(net))

    # Remove the bounding boxes with low confidence
    faces = post_process(frame, outs, CONF_THRESHOLD, NMS_THRESHOLD)
    print('[i] ==> # detected faces: {}'.format(len(faces)))
    print('#' * 60)

    # initialize the set of information we'll displaying on the frame
    info = [
        ('number of faces detected', '{}'.format(len(faces)))
    ]

    for face in faces:
        left, top, width, height = face[0], face[1], face[2], face[3]
        startX, startY, endX, endY = left, top, left+width, top+height
        face = frame[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # pass the face through the model to determine if the face
        # has a mask or not
        (notCorrect, mask, withoutMask) = model.predict(face)[0]

        # determine the class label and color we'll use to draw
        # the bounding box and text
        if notCorrect > max(mask, withoutMask):
             label, color = "Not correct", (0, 255, 255)
        elif mask > max (withoutMask, notCorrect):
            label, color = "Mask", (0, 255,0)
        elif withoutMask > max (mask, notCorrect):
            label, color = "No Mask", (0, 0, 255)
        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX-30, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.namedWindow("LIVE", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("LIVE",cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("LIVE", frame)
    key = cv2.waitKey(10)
    if key==27:
        break
# Stop video
cap.release()

# Close all started windows
cv2.destroyAllWindows()


[i] ==> # detected faces: 0
############################################################
[i] ==> # detected faces: 0
############################################################
[i] ==> # detected faces: 0
############################################################
[i] ==> # detected faces: 0
############################################################
[i] ==> # detected faces: 1
############################################################
[i] ==> # detected faces: 1
############################################################
[i] ==> # detected faces: 1
############################################################
[i] ==> # detected faces: 1
############################################################
